# ABS Monthly Consumer Price Index Indicator 6484

## Python set-up

In [1]:
# analytic imports
import matplotlib.pyplot as plt
import pandas as pd

# local imports
from abs_data_capture import (
    find_id,
    get_fs_constants,
    metacol,
    get_plot_constants,
    AbsLandingPage,
    get_abs_data,
)
from henderson import hma
from plotting import (
    COLOR_AMBER,
    COLOR_BLUE,
    COLOR_RED,
    calc_growth,
    clear_chart_dir,
    finalise_plot,
    line_plot,
    plot_growth,
    set_chart_dir,
)
from utility import annualise_rates

# pandas display settings
pd.options.display.max_rows = 999999
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 100

# display charts in this notebook
SHOW = False

## Get data from ABS

In [2]:
landing_page = AbsLandingPage(
    theme="economy",
    parent_topic="price-indexes-and-inflation",
    topic="monthly-consumer-price-index-indicator",
)
abs_dict = get_abs_data(landing_page)
source, CHART_DIR, Cat_ID, meta = get_fs_constants(abs_dict, landing_page)
RECENT, plot_times, plot_tags = get_plot_constants(meta)

Retrieving data from cache.
Extracting DataFrames from the zip-file.


## Plot

### Set-up 

In [3]:
def chart_settings(chart_dir: str = CHART_DIR) -> None:
    """Set up charting parameters."""

    set_chart_dir(chart_dir)
    clear_chart_dir(chart_dir)
    plt.style.use("fivethirtyeight")


chart_settings()
TAREGT_COLOR = "#dddddd"

### Plot growth - combined line/bar charts

In [4]:
def cpi_growth():
    """Produce CPI growth charts."""

    rows = meta[meta[metacol.did].str.contains("Index Numbers")]

    for index, row in rows.iterrows():
        # get data for the plot
        did, table = row[metacol.did], row[metacol.table]
        series_type, series_id = row[metacol.stype], row[metacol.id]
        name = did.split(";")[1].strip()
        index = abs_dict[table][series_id]
        growth = calc_growth(index)

        # plot the data
        ax = plot_growth(*growth)
        if ax:
            ax.axhspan(
                2, 3, color=TAREGT_COLOR, zorder=-1, label="Annual inflation target"
            )
            ax.legend(loc="best", fontsize="small")
            long_name = 40
            separator = " " if len(name) < long_name else "\n"
            finalise_plot(
                ax,
                title=f"Price Growth:{separator}{name}",
                ylabel="Per cent",
                rfooter=source,
                lfooter=f"Australia: {series_type.capitalize()} series. ",
                show=SHOW,
            )

In [5]:
cpi_growth()

### Annualised growth

In [6]:
def line_and_target_plot(series: pd.Series, **kwargs) -> None:
    """Finish up plot."""

    ax = series.plot(lw=2, color=COLOR_RED)
    ax.axhspan(2, 3, color=TAREGT_COLOR, zorder=-1, label="Annual inflation target")
    finalise_plot(ax, **kwargs)


def annualised_growth() -> None:
    """Produce annualised growth charts."""

    henderson = 13

    for series_name in (
        "Index Numbers ;  All groups CPI ;  Australia ;",
        "Index Numbers ;  All groups CPI, seasonally adjusted ;  Australia ;",
    ):
        print(series_name)
        table = "1"
        selector = {series_name: metacol.did}
        series_id, _ = find_id(meta, selector, verbose=False)
        series_type = (
            "Seasonally Adjusted"
            if "seasonally adjusted" in series_name
            else "Original"
        )
        name = series_name.split(";")[1].strip()
        long_name = 20
        separator = " " if len(name) < long_name else "\n"
        name = f"{separator}{name}"
        lfooter = f"Australia. {series_type.capitalize()} series. "

        quarterly = annualise_rates(
            abs_dict[table][series_id].pct_change(3), 4
        ).dropna()
        print(f"Quarterly annualised endpoint: {quarterly.iloc[-1]:0.2f}%")
        line_and_target_plot(
            quarterly,
            title=f"Quarterly Price Growth Annualised:{name}",
            ylabel="Per cent",
            lfooter=lfooter,
            rfooter=source,
            show=SHOW,
        )

        quarterly_s = hma(quarterly, henderson)
        print(f"Quarterly annualised & smoothed endpoint: {quarterly_s.iloc[-1]:0.2f}%")
        line_and_target_plot(
            quarterly_s,
            title=f"Quarterly Price Growth Annualised/Smoothed:{name}",
            ylabel="Per cent",
            lfooter=f"{lfooter}{henderson}-term Henderson moving average. ",
            rfooter=source,
            show=SHOW,
        )

        monthly = annualise_rates(abs_dict[table][series_id].pct_change(1), 12).dropna()
        print(f"Monthly annualised endpoint: {monthly.iloc[-1]:0.2f}%")
        line_and_target_plot(
            monthly,
            title=f"Monthly Price Growth Annualised:{name}",
            ylabel="Per cent",
            lfooter=lfooter,
            rfooter=source,
            show=SHOW,
        )

        monthly_s = hma(monthly, henderson)
        print(f"Monthly annualised & smoothed endpoint: {monthly_s.iloc[-1]:0.2f}%")
        line_and_target_plot(
            monthly_s,
            title=f"Monthly Price Growth Annualised/Smoothed:{name}",
            ylabel="Per cent",
            lfooter=f"{lfooter}{henderson}-term Henderson moving average. ",
            rfooter=source,
            show=SHOW,
        )
        print()

In [7]:
annualised_growth()

Index Numbers ;  All groups CPI ;  Australia ;
Quarterly annualised endpoint: 2.66%
Quarterly annualised & smoothed endpoint: 3.08%
Monthly annualised endpoint: 8.20%
Monthly annualised & smoothed endpoint: 3.93%

Index Numbers ;  All groups CPI, seasonally adjusted ;  Australia ;
Quarterly annualised endpoint: 1.32%
Quarterly annualised & smoothed endpoint: 3.04%
Monthly annualised endpoint: -0.98%
Monthly annualised & smoothed endpoint: 1.06%



### Selected sub items

In [8]:
def food_fuel() -> None:
    """Produce food and fuel sub-item charts."""

    annual = "Corresponding Month of Previous Year"
    selectors = {
        "Food and non-alcoholic beverages": {
            "Food and non-alcoholic beverages": metacol.did,
            "Original": metacol.stype,
            annual: metacol.did,
        },
        "Automotive fuel": {
            "Automotive fuel": metacol.did,
            "Original": metacol.stype,
            annual: metacol.did,
        },
    }

    data = pd.DataFrame()
    for name, selector in selectors.items():
        series_id, _ = find_id(meta, selector, verbose=False)
        table = meta.loc[meta["Series ID"] == series_id, "Table"].values[0]
        data[name] = abs_dict[table][series_id]

    title = "Monthly CPI Indicator - Annual Growth - Food & Fuel"
    tag = "1"
    series_type = "Original series"
    line_plot(
        data,
        title=title,
        ylabel="Per cent",
        color=[COLOR_BLUE, COLOR_AMBER],
        width=1.5,
        tags=tag,
        legend={"loc": "best", "fontsize": "x-small"},
        rfooter=f"{source}",
        lfooter=f"Australia: {series_type}",
        show=SHOW,
    )


food_fuel()

In [9]:
def housing() -> None:
    """Produce housing related charts."""

    annual = "Corresponding Month of Previous Year"
    selectors = {
        "Rents": {
            "Rents": metacol.did,
            "Original": metacol.stype,
            annual: metacol.did,
        },
        "New dwelling purchase by owner-occupiers": {
            "New dwelling purchase by owner-occupiers": metacol.did,
            "Original": metacol.stype,
            annual: metacol.did,
        },
    }

    data = pd.DataFrame()
    for name, selector in selectors.items():
        series_id, _ = find_id(meta, selector, verbose=False)
        table = meta.loc[meta[metacol.id] == series_id, "Table"].values[0]
        data[name] = abs_dict[table][series_id]

    title = "Monthly CPI Indicator - Annual Growth - Residential"
    tag = "2"
    series_type = "Original series"
    line_plot(
        data,
        title=title,
        ylabel="Per cent",
        color=[COLOR_BLUE, COLOR_AMBER],
        width=1.5,
        tags=tag,
        legend={"loc": "best", "fontsize": "x-small"},
        rfooter=f"{source}",
        lfooter=f"Australia: {series_type}",
        show=SHOW,
    )


housing()

## Finished

In [10]:
# watermark
%load_ext watermark
%watermark -u -n -t -v -iv -w

Last updated: Wed Feb 21 2024 13:23:11

Python implementation: CPython
Python version       : 3.11.8
IPython version      : 8.21.0

matplotlib: 3.8.3
pandas    : 2.2.0

Watermark: 2.4.3



In [11]:
print("Done")

Done
